In [3]:
import os
import librosa
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize Spark Session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Audio Classification with RandomForest") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Base directory for audio files
base_dir = os.path.expanduser('~/Downloads/data')
directories = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

# Function to extract MFCC features
def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    return mfcc.mean(axis=1).tolist()

# Load and preprocess data
audio_features = []
for directory in directories:
    genre = directory
    path = os.path.join(base_dir, directory)
    audio_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.wav')]
    for file in audio_files:
        features = extract_features(file)
        audio_features.append((genre, file, features))

# Create DataFrame
df_audio = pd.DataFrame(audio_features, columns=['genre', 'file_path', 'audio_features'])
sdf_audio = spark.createDataFrame(df_audio)

# Convert array of doubles to Vector
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
sdf_audio = sdf_audio.withColumn("features", list_to_vector_udf("audio_features"))

# Index labels
indexer = StringIndexer(inputCol="genre", outputCol="label")
sdf_audio = indexer.fit(sdf_audio).transform(sdf_audio)

# Split the data
train_data, test_data = sdf_audio.randomSplit([0.8, 0.2])

# Define and train the Random Forest model
rf = RandomForestClassifier(featuresCol='features', labelCol='label', numTrees=20, maxDepth=5, seed=42)
model = rf.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Evaluate the model
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

accuracy = evaluator_accuracy.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)
f1_score = evaluator_f1.evaluate(predictions)

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1_score)

spark.stop()


24/04/20 14:37:39 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.nio.file.NoSuchFileException: /home/eashwar123/Desktop/sem4/Bigdata/project/venv/lib/python3.10/site-packages/pyspark/jars/hadoop-client-api-3.3.4.jar
	at org.apache.hadoop.conf.Configuration.loadResource(Configuration.java:3089)
	at org.apache.hadoop.conf.Configuration.loadResources(Configuration.java:3036)
	at org.apache.hadoop.conf.Configuration.loadProps(Configuration.java:2914)
	at org.apache.hadoop.conf.Configuration.getProps(Configuration.java:2896)
	at org.apache.hadoop.conf.Configuration.set(Configuration.java:1412)
	at org.apache.spark.deploy.SparkHadoopUtil$.org$apache$spark$deploy$SparkHadoopUtil$$appendSparkHadoopConfigs(SparkHadoopUtil.scala:512)
	at org.apache.spark.deploy.SparkHadoopUtil$.org$apache$spark$deploy$SparkHadoopUtil$$appendS3AndSparkHadoopHiveConfigurations(SparkHadoopUtil.scala:443)
	at org.apache.spark.deploy.SparkHadoopUtil$.n